# Integración y preparación de datos

## Estandarización 

### 1. Importación de librerías y archivos

En las siguientes líneas se importarán las librerías necesarias, las cuales son **Pandas** para el manejo de datos.

In [1]:
# Importar las librerías necesarias según el análisis que se vaya a realizar
# Librería para manejo de datos convensional
import pandas as pd

In [23]:
# cargar los datos en csv
data= pd.read_csv('productos_por_fecha.csv', sep=';', encoding='latin-1')
# Visualizar los datos
data.head()

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto,volumen_cm3_producto
0,PA53,9-may-18,Electrodomésticos,19,28,29,48,17,11,8976
1,PD80,18-oct-17,Celulares,18,29,16,26,6,16,2496
2,PB61,10-sep-17,Ropa_de_adultos,13,30,29,17,26,24,10608
3,PS63,1/11/2018,Carnicería,33,26,13,47,16,43,32336
4,PA71,17/11/2017,Electrodomésticos,27,3,13,20,26,28,14560


### 2. Análisis de la fuente de los datos

### 2.1. Registros y atributos
El número de registros y atributos nos dará una guía del correcto cargue de los datos, podemos utilizar la función shape para lograr este objetivo.

In [3]:
# Tamaño del dataset (filas, columnas)
data.shape

(1161, 10)

In [4]:
# Revisión de los atributos
data.columns

Index(['producto_id', 'fecha_envio_limite', 'nombre_categoria_producto',
       'longitud_nombre_producto', 'longitud_descripcion_producto',
       'cantidad_fotos_producto', 'longitud_cm_producto', 'altura_cm_producto',
       'ancho_cm_producto', 'volumen_cm3_producto'],
      dtype='object')

### 2.2. Tipos de datos

Debemos identificar en la fuente el tipo de datos de las variables que estamos analizando, para poder establecer su formato y su dominio. Recordemos en este punto lo importante de corroborarlos con el diccionario de datos de la fuente.

In [5]:
#identificar los tipos de datos
data.dtypes

producto_id                      object
fecha_envio_limite               object
nombre_categoria_producto        object
longitud_nombre_producto          int64
longitud_descripcion_producto     int64
cantidad_fotos_producto           int64
longitud_cm_producto              int64
altura_cm_producto                int64
ancho_cm_producto                 int64
volumen_cm3_producto              int64
dtype: object

En nuestra fuente de datos tenemos tipo de datos numérico y string, esto corresponde con las características del producto y los datos que se especifican en el diccionario, a excepción de los primeros identificadores y del atributo fecha_envio_limite que debemos transformar a Datetime.

In [6]:
#describir el dominio de los atributos y otras características estadísticas
data.describe()

,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto,volumen_cm3_producto
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,20.046512,20.046512,16.951766,29.891473,18.114556,23.364341,9549.582257
std,11.687876,11.663585,14.782367,14.960878,11.331516,10.993293,14323.671275
min,0.000000,0.000000,-39.000000,0.000000,0.000000,0.000000,-90828.000000
25%,10.000000,10.000000,7.000000,19.000000,9.000000,16.000000,1080.000000
50%,20.000000,20.000000,18.000000,30.000000,17.000000,23.000000,5814.000000
75%,30.000000,30.000000,28.000000,41.000000,26.000000,31.000000,14868.000000
max,40.000000,40.000000,40.000000,76.000000,59.000000,62.000000,78925.000000


Con este resumen de los datos, podemos identificar rápidamente el dominio de atributos de caracter numérico lo cuales deberían cumplir ciertos requisitos.

### 3. Estandarización 

### 3.1. Estandarizar un atributo tipo fecha
Inicialmente identificamos que en la fuente de datos el valor del atributo es de tipo string, por lo que debemos transformarlo a datetime, sin embargo, primero debemos revisar el formato tiene.

In [7]:
# revisar el formato del atributo  fecha_envío_límite
data[['fecha_envio_limite']].head(20)

,fecha_envio_limite
0,9-may-18
1,18-oct-17
2,10-sep-17
3,1/11/2018
4,17/11/2017
5,6-nov-01
6,20/09/2017
7,10/25/2017
8,02/14/2018
9,01-14-2018


debemos generar una función para transformar los datos al formato estándar de la mayoría de registros que es: **dd/mm/AAAA hh:mm**

Esto lo logramos entender cuando vemos los 3 formatos utilizados para la fecha e identificamos el formato más frecuente, lo cual se se hace en un proceso manual. Al aplicarlo a este caso, identificamos los siguientes patrones:

* dd - mes - AA
* dd/mm/AAAA
* dd/mm/AAAA hh:mm

Con la identificación de los patrones estamos listos para continuar. La estrategia es crear dos funciones adicionales que van a aportar en la identificación del mes asociado a la fecha y en el caracter utilizado como separador.

In [8]:
# función para pasar el mes que están en letras a números
def mes_a_numero(mes):
    numero=''
    if mes=='ene':
        numero='01'
    if mes=='feb':
        numero='02'
    if mes=='mar':
        numero='03'
    if mes=='abr':
        numero='04'
    if mes=='may':
        numero='05'
    if mes=='jun':
        numero='06'
    if mes=='jul':
        numero='07'
    if mes=='ago':
        numero='08'
    if mes=='sep':
        numero='09'
    if mes=='oct':
        numero='10'
    if mes=='nov':
        numero='11'
    if mes=='dic':
        numero='12'
    return numero
        
# función para validar que es correcto el separador utilizado para la fecha
def validar_corregir_fecha(fecha):
    # revisar que el guión es el separador
    if '-' in fecha:
        vector_fecha=fecha.split('-')
        # si el primer registro es el año sabemos cómo rearmar el formato
        if len(vector_fecha[0])==4:
            fecha_nueva= vector_fecha[2]+'/'+vector_fecha[1]+'/'+vector_fecha[0]+' 00:00' # consstruir el formato adecuado
        else:
            fecha_nueva= vector_fecha[0]+'/'+mes_a_numero(vector_fecha[1])+'/20'+vector_fecha[2]+' 00:00'
    # si el guión no está en el registro el formato es correcto
    else:
        if ' ' in fecha:
            fecha_nueva=fecha
        else:
            vector_fecha = fecha.split('/')
            fecha_nueva = vector_fecha[0]+'/'+vector_fecha[1]+'/'+vector_fecha[2]+' 00:00'
    return fecha_nueva

In [9]:
# validamos y si es necesario corregimos el formato de la fecha con la función validar_corregir_fecha
print(data.at[0,'fecha_envio_limite'])
validar_corregir_fecha(data.at[0,'fecha_envio_limite'])


9-may-18


'9/05/2018 00:00'

In [10]:
# validamos y si es necesario corregimos el formato de la fecha con la función validar_corregir_fecha
print(data.at[3,'fecha_envio_limite'])
validar_corregir_fecha(data.at[3,'fecha_envio_limite'])


1/11/2018


'1/11/2018 00:00'

In [11]:
# validamos y si es necesario corregimos el formato de la fecha con la función validar_corregir_fecha
print(data.at[5,'fecha_envio_limite'])
validar_corregir_fecha(data.at[5,'fecha_envio_limite'])


6-nov-01


'6/11/2001 00:00'

In [20]:
# validamos y si es necesario corregimos el formato de la fecha con la función validar_corregir_fecha
print(data.at[10,'fecha_envio_limite'])
validar_corregir_fecha(data.at[10,'fecha_envio_limite'])

3/10/1901 11:00


'3/10/1901 11:00'

Vemos cómo los registros 0, 3 y 5 que son los que tienen los 3 formatos distintos se pueden transformar fácilmente con la función creada, por lo tanto podemos proceder a automatizar y aplicar la función sobre toda la fuente de datos.

In [13]:
# se aplica la función a todo el atributo de fecha
data['fecha_envio_limite']=data['fecha_envio_limite'].apply(validar_corregir_fecha)
data[['fecha_envio_limite']].head(30)

,fecha_envio_limite
0,9/05/2018 00:00
1,18/10/2017 00:00
2,10/09/2017 00:00
3,1/11/2018 00:00
4,17/11/2017 00:00
5,6/11/2001 00:00
6,20/09/2017 00:00
7,10/25/2017 00:00
8,02/14/2018 00:00
9,01//202018 00:00


In [24]:
# convertir a datetime el atributo fecha_envio_limite
#data['fecha_envio_limite']=pd.to_datetime(data['fecha_envio_limite'])
#data[['fecha_envio_limite']].head()

#Se produce error.

In [14]:
# revisar que el tipo de dato haya quedado correcto
data[['fecha_envio_limite']].dtypes

fecha_envio_limite    object
dtype: object

Pudimos ver que el proceso de encontrar el estándar para cada variable puede ser costoso en tiempo, en especial este tipo de campos como el de fecha, cuyo contenido se puede escribir de varias formas aunque represente lo mismo. Sin embargo, con la revisión manual y funciones automatizadas se puede lograr tener el contenido del dato correcto. 

### 3.2. Estandarizar un atributo categórico

In [25]:
# Consultar las categorías existentes
data['nombre_categoria_producto'].value_counts()

Electrodomésticos    171
Deportes             164
Carnicería           161
Celulares            155
Frutas y verduras    153
Ropa de adultos      152
C4rnicerí4            18
Ropa_de_adultos       17
Frut4s_y_verdur4s     14
Rop4_de_4dultos        7
Frutas_y_verduras      7
n4n                    6
Celul4res              4
Name: nombre_categoria_producto, dtype: int64

In [26]:
# función para incluir el guión bajo
def transformar_categoria(nombre_categoria):
    nuevo_nombre=str(nombre_categoria).replace('_',' ') # reemplazamos los espacios en blanco por guión bajo
    nuevo_nombre=str(nuevo_nombre).replace('4','a')
    return nuevo_nombre

In [27]:
# aplicamos la función a todos los registros del atributo
data['nombre_categoria_producto']=data['nombre_categoria_producto'].apply(transformar_categoria)
data['nombre_categoria_producto']

0       Electrodomésticos
1               Celulares
2         Ropa de adultos
3              Carnicería
4       Electrodomésticos
              ...        
1156           Carnicería
1157      Ropa de adultos
1158      Ropa de adultos
1159                  nan
1160           Carnicería
Name: nombre_categoria_producto, Length: 1161, dtype: object

In [28]:
# validamos las categorías
data['nombre_categoria_producto'].value_counts()

Carnicería           179
Ropa de adultos      176
Frutas y verduras    174
Electrodomésticos    171
Deportes             164
Celulares            159
nan                  138
Name: nombre_categoria_producto, dtype: int64

Podemos ver que en este último caso, también encontramos campos vacíos o mal escritos. Esto lo vamos a corregir en una siguiente lección.